In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm, uniform

In [2]:
def negative_log_posterior(params, data, prior_mean, prior_std):
    mu, sigma = params

    # Log-likelihood (e.g., for a normal distribution)
    log_likelihood = np.sum(norm.logpdf(data, loc=mu, scale=sigma))

    # Log-prior (e.g., for mu and sigma)
    log_prior_mu = norm.logpdf(mu, loc=prior_mean, scale=prior_std)
    # You might use a different prior for sigma, e.g., an inverse-gamma
    # For simplicity, using a uniform prior here or another normal for demonstration
    log_prior_sigma = uniform.logpdf(sigma, loc=0.01, scale=10) # Example uniform prior

    return -(log_likelihood + log_prior_mu + log_prior_sigma)


# Example data and prior parameters
data = np.random.normal(loc=5, scale=2, size=100)
prior_mean = 4
prior_std = 1

# Initial guess for parameters (mu, sigma)
initial_guess = [np.mean(data), np.std(data)]

# Perform the optimization
result = minimize(negative_log_posterior, initial_guess, 
                  args=(data, prior_mean, prior_std), 
                  method='L-BFGS-B', 
                  bounds=[(None, None), (0.001, None)]) # Bounds for sigma

map_estimates = result.x
print(f"MAP Estimates: mu = {map_estimates[0]:.2f}, sigma = {map_estimates[1]:.2f}")

MAP Estimates: mu = 5.45, sigma = 2.18


In [3]:
# 1. 正規分布の平均のMAP推定（正規事前分布）

In [4]:
import numpy as np
from scipy import optimize
from scipy import stats

# データ生成
np.random.seed(42)
true_mu = 5.0
data = np.random.normal(true_mu, 1.0, size=20)

# 事前分布のパラメータ（正規分布）
prior_mu = 0.0
prior_sigma = 10.0

# 既知の標準偏差
sigma = 1.0

def neg_log_posterior(mu):
    # 対数尤度
    log_likelihood = np.sum(stats.norm.logpdf(data, loc=mu, scale=sigma))
    # 対数事前分布
    log_prior = stats.norm.logpdf(mu, loc=prior_mu, scale=prior_sigma)
    return -(log_likelihood + log_prior)

# MAP推定
result = optimize.minimize(neg_log_posterior, x0=0.0, method='BFGS')
print(f"MAP推定値: {result.x[0]:.4f}")
print(f"MLE（単純平均）: {np.mean(data):.4f}")
print(f"真の値: {true_mu}")

MAP推定値: 4.8263
MLE（単純平均）: 4.8287
真の値: 5.0


In [5]:
# 2. ベルヌーイ分布のMAP推定（ベータ事前分布）

In [6]:
import numpy as np
from scipy import optimize
from scipy import stats

# データ: コイン投げ（1=表, 0=裏）
data = np.array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1])

# ベータ事前分布のパラメータ
alpha_prior = 2.0
beta_prior = 2.0

def neg_log_posterior(p):
    p = p[0]
    if p <= 0 or p >= 1:
        return np.inf
    
    # 対数尤度（ベルヌーイ）
    log_likelihood = np.sum(data * np.log(p) + (1 - data) * np.log(1 - p))
    # 対数事前分布（ベータ）
    log_prior = stats.beta.logpdf(p, alpha_prior, beta_prior)
    return -(log_likelihood + log_prior)

# MAP推定
result = optimize.minimize(neg_log_posterior, x0=[0.5], method='L-BFGS-B', 
                          bounds=[(1e-6, 1-1e-6)])

# 解析解との比較
n_success = np.sum(data)
n_total = len(data)
analytical_map = (n_success + alpha_prior - 1) / (n_total + alpha_prior + beta_prior - 2)

print(f"MAP推定値（数値解）: {result.x[0]:.4f}")
print(f"MAP推定値（解析解）: {analytical_map:.4f}")
print(f"MLE: {n_success / n_total:.4f}")

MAP推定値（数値解）: 0.6667
MAP推定値（解析解）: 0.6667
MLE: 0.7000


In [7]:
# 3. 線形回帰のMAP推定（L2正則化 = リッジ回帰）

In [8]:
import numpy as np
from scipy import optimize

# データ生成
np.random.seed(42)
n_samples, n_features = 50, 3
X = np.random.randn(n_samples, n_features)
true_weights = np.array([1.5, -2.0, 0.5])
y = X @ true_weights + np.random.randn(n_samples) * 0.5

# ハイパーパラメータ
sigma_noise = 0.5  # ノイズの標準偏差
sigma_prior = 1.0  # 重みの事前分布の標準偏差（正規分布）

def neg_log_posterior(w):
    # 対数尤度（正規分布のノイズ）
    residuals = y - X @ w
    log_likelihood = -0.5 * np.sum(residuals**2) / sigma_noise**2
    
    # 対数事前分布（重みに正規分布）
    log_prior = -0.5 * np.sum(w**2) / sigma_prior**2
    
    return -(log_likelihood + log_prior)

# MAP推定
result = optimize.minimize(neg_log_posterior, x0=np.zeros(n_features), method='BFGS')

# sklearn のリッジ回帰と比較
from sklearn.linear_model import Ridge
alpha = (sigma_noise / sigma_prior) ** 2
ridge = Ridge(alpha=alpha, fit_intercept=False).fit(X, y)

print("MAP推定値:", result.x)
print("Ridge回帰:", ridge.coef_)
print("真の重み:", true_weights)

MAP推定値: [ 1.54813273 -2.06661518  0.49211504]
Ridge回帰: [ 1.54813275 -2.06661517  0.49211504]
真の重み: [ 1.5 -2.   0.5]


In [9]:
# 4. 汎用的なMAPクラス

In [10]:
import numpy as np
from scipy import optimize
from scipy import stats
from typing import Callable

class MAPEstimator:
    """汎用MAP推定クラス"""
    
    def __init__(self, log_likelihood_fn: Callable, log_prior_fn: Callable):
        self.log_likelihood_fn = log_likelihood_fn
        self.log_prior_fn = log_prior_fn
        self.result_ = None
    
    def fit(self, x0, bounds=None, method='L-BFGS-B'):
        def neg_log_posterior(theta):
            ll = self.log_likelihood_fn(theta)
            lp = self.log_prior_fn(theta)
            return -(ll + lp)
        
        self.result_ = optimize.minimize(
            neg_log_posterior, x0=x0, method=method, bounds=bounds
        )
        return self
    
    @property
    def theta_map(self):
        return self.result_.x if self.result_ else None


# 使用例: ポアソン分布のλのMAP推定（ガンマ事前分布）
data = np.array([3, 5, 4, 6, 2, 4, 5, 3, 4, 5])

# ガンマ事前分布: α=2, β=1
alpha_prior, beta_prior = 2.0, 1.0

log_likelihood = lambda lam: np.sum(stats.poisson.logpmf(data, mu=lam[0]))
log_prior = lambda lam: stats.gamma.logpdf(lam[0], a=alpha_prior, scale=1/beta_prior)

estimator = MAPEstimator(log_likelihood, log_prior)
estimator.fit(x0=[1.0], bounds=[(1e-6, None)])

print(f"ポアソンλのMAP推定: {estimator.theta_map[0]:.4f}")
print(f"MLE（標本平均）: {np.mean(data):.4f}")

ポアソンλのMAP推定: 3.8182
MLE（標本平均）: 4.1000
